In [5]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv('training_data.csv')
test = pd.read_csv('test_data.csv')
#print(test)
#print(df)
outid = test['review_id'].values # dataframe to numpy
#print(outid.T)
#print(len(outid))
train = df

In [6]:
from sklearn.svm import LinearSVC

X_train = df.drop(['review_id','business_id','user_id','date'], axis=1) 

# use tfidf 
vectorizer = TfidfVectorizer()#stop_words="english",min_df = 0.01) 
vectorizer.fit(train['text'])

X_train = vectorizer.transform(train['text'])
X_test = vectorizer.transform(test['text'])
y_train = train['stars'].as_matrix()

# use the linearSVC model
clf = LinearSVC() # default = 1000
clf = clf.fit(X_train, y_train)
pred_y = clf.predict(X_test)

ans1 = pred_y
print(clf.score(X_train, y_train))

0.9707390271351757


In [7]:
from sklearn.svm import LinearSVR

# set ngram the 1 to 3 combine words will be counted 
vectorizer = TfidfVectorizer(ngram_range=[1,3])#,max_df = 2000)#,min_df = 0.01)#,stop_words="english",min_df = 0.009) 
vectorizer.fit(train['text'])

X_train = vectorizer.transform(train['text'])
X_test = vectorizer.transform(test['text'])
y_train = train['stars'].as_matrix()

# use linearSVR model
clf2 = LinearSVR() # default = 1000
clf2 = clf2.fit(X_train, y_train)
pred_y = clf2.predict(X_test)

print(clf2.score(X_train, y_train))

# cause the linearSVR model is linear so the answer will out of range(1 to 5) 
# set the answer which is more than 5 to 5
for i in range(0,len(pred_y)):
    if(pred_y[i]>5):
        pred_y[i] = 5.0
    elif(pred_y[i]<1):
        pred_y[i] = 1.0
    else:
        pred_y[i] = round(pred_y[i],2)

ans2 = pred_y

0.9087621043235661


In [9]:
# answer 0.80710.5042

# get the answer from linearSVC & linearSVR
# set the weighted 3:8
new = (ans1*3+ans2*8)/11

# to get the lower RMSE & higher accuracy
# reshape the range of the answer to 1.5~4.6 (not 1~5)
for i in range(0,len(new)):
    if(new[i]>=4.5):#4.4
        new[i] = 4.6
    elif(new[i]<1.5):
        new[i] = 1.5
    else:
        new[i] = round(new[i],3)

output = np.vstack((outid,new)).T
output = pd.DataFrame(output,columns=['user_id','stars'])
output['user_id'] = output['user_id'].astype(int)

# the answer is stored in the answer.csv
np.savetxt("answer.csv", output, fmt='%i, %1.4f', delimiter=",")